In [1]:
print("hello")
# Go here https://python.langchain.com/v0.1/docs/get_started/quickstart/ and use the web scraping stuff to pass in the Vice article

hello


In [3]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.document_loaders import WebBaseLoader
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.documents import Document
from langchain.chains import create_retrieval_chain



In [4]:
embeddings = OpenAIEmbeddings()
loader = WebBaseLoader("https://www.vice.com/en/article/k7zz9z/spotify-rogan-rfk-vaccine-misinformation-policy")
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)
vector = FAISS.from_documents(documents, embeddings)


In [5]:
"""
output_parser = StrOutputParser()
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a world class technical documentation writer."),
    ("user", "{input}")
])
chain = prompt | llm | output_parser
a = chain.invoke({"input": "how can langsmith help with testing?"})
"""

llm = ChatOpenAI()

prompt = ChatPromptTemplate.from_template("""Answer the following question by referencing the provided context:

<context>
{context}
</context>

Question: {input}""")

document_chain = create_stuff_documents_chain(llm, prompt)
"""
# Pass context directly into 'page_content'. The llm responds based only on the context provided in the page_content
a = document_chain.invoke({
    "input": "how can langsmith help with testing?",
    "context": [Document(page_content="langsmith can let you create incredible videos visualize test results")]
})
"""
retriever = vector.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

# Note -- 
response = retrieval_chain.invoke({"input": "List the scientific papers referenced in the provided article. Provide them in a numbered list."})
print(response["answer"])

# LangSmith offers several features that can help with testing:...


1. American Academy of Pediatrics roundup of safety studies on vaccines
2. Large Australian study on COVID vaccinations and cardiac arrest

Please note that no specific scientific papers were directly referenced in the context provided.
